In [1]:
import numpy as np
from scipy.stats import mode
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from warnings import filterwarnings
filterwarnings('ignore')
import os
import cv2

In [2]:
import numpy as np
from scipy.stats import mode
import seaborn as sns
sns.set_style('white')
import matplotlib.pyplot as plt
%matplotlib inline
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
import theano
import pymc3 as pm
import lasagne
import theano.tensor as T

floatX = theano.config.floatX

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GRID K520 (CNMeM is disabled, cuDNN 5005)


In [22]:
import theano
import pymc3 as pm
import lasagne
from lasagne.utils import floatX
from lasagne.layers import InputLayer, DenseLayer, NonlinearityLayer, DropoutLayer
from lasagne.layers import Pool2DLayer as PoolLayer
from lasagne.layers import Conv2DLayer as ConvLayer
from lasagne.nonlinearities import rectify, softmax
import theano.tensor as T

floatX = theano.config.floatX

In [23]:
import sklearn
from sklearn import datasets
from sklearn.preprocessing import scale
from sklearn.cross_validation import train_test_split
from sklearn.datasets import fetch_mldata
from sklearn.metrics import accuracy_score

In [24]:
def preprocess(img, h=64, w=64):
    img = img.astype('float32')
    
    img_resized = np.zeros(shape=(1, 3, h, w))
    img_resized[0, 2, :, :] = cv2.resize(img[:, :, 0] - MEAN_VALUES[2], (h, w))
    img_resized[0, 1, :, :] = cv2.resize(img[:, :, 1] - MEAN_VALUES[1], (h, w)) 
    img_resized[0, 0, :, :] = cv2.resize(img[:, :, 2] - MEAN_VALUES[0], (h, w)) 
    
    return img_resized.astype('float32')

In [25]:
dir_path = 'data/train_subset/'

file_names = os.listdir(dir_path)
n_objects = len(file_names)

y_train = []
X_train = np.zeros(shape=(n_objects, 3, 64, 64), dtype='float32')

MEAN_VALUES = [0, 0, 0]
for i, f in enumerate(file_names):
    img = scipy.misc.imread(dir_path + f)
    y_train.append(f[:3] == 'cat')
    MEAN_VALUES[2] += img[:, :, 0].mean()
    MEAN_VALUES[1] += img[:, :, 1].mean()
    MEAN_VALUES[0] += img[:, :, 2].mean()

MEAN_VALUES = 1. * np.array(MEAN_VALUES) / n_objects
y_train = np.array(y_train, dtype='int32')

for i, f in enumerate(file_names):
    img = scipy.misc.imread(dir_path + f)
    X_train[i] = preprocess(img)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3)

In [27]:
input_shape = list(X_train.shape)
input_shape[0] = None
input_shape = tuple(input_shape)
input_shape

(None, 3, 64, 64)

In [29]:
class PriorWeights(object):
    def __init__(self, mode='W', prior='gauss', **params):
        self.count = 0
        self.prior = prior
        self.mode = mode
        params.setdefault('std', 1.)
        params.setdefault('hyper', None)
        self.params = params
    def __call__(self, shape):
        self.count += 1
        if self.params['hyper'] is None:
            std = self.params['std']
        elif self.params['hyper'] == 'cauchy':
            std = pm.HalfCauchy('hyper_%s%d' % (self.mode, self.count), beta=1.)
        elif self.params['hyper'] == 'normal':
            std = pm.HalfNormal('hyper_%s%d' % (self.mode, self.count), mu=0., sd=1.)
        elif self.params['hyper'] == 'flat':
            std = pm.Flat()
        if self.prior == 'gauss':
            return pm.Normal('%s%d' % (self.mode, self.count), mu=0, sd=std, 
                         testval=np.random.normal(size=shape).astype(np.float64),
                         shape=shape)
        elif self.prior == 'laplace': 
            return pm.Laplace('%s%d' % (self.mode, self.count), mu=0, b=std, 
                         testval=np.random.normal(size=shape).astype(np.float64),
                         shape=shape)
        elif self.prior == 'flat':
            return pm.Flat('%s%d' % (self.mode, self.count), 
                           testval=np.random.normal(size=shape).astype(np.float64), 
                           shape=shape)

In [30]:
def build_ann(prior_b, prior_W, input_var, target_var, 
              input_shape, params=[176, 64, 10]):
    
    n_hid1, n_hid2, n_classes = params
    with pm.Model() as neural_network:
        l_in = lasagne.layers.InputLayer(shape=input_shape,
                                         input_var=input_var)
        l_conv1 = lasagne.layers.Conv2DLayer(
                            l_in, num_filters=32, filter_size=(5, 5),
                            b=prior_b, W=prior_W,
                            nonlinearity=lasagne.nonlinearities.tanh)
        
        l_pool1 = lasagne.layers.MaxPool2DLayer(l_conv1, pool_size=(2, 2))
        
        l_conv2 = lasagne.layers.Conv2DLayer(
                            l_pool1, num_filters=32, filter_size=(5, 5),
                            b=prior_b, W=prior_W,
                            nonlinearity=lasagne.nonlinearities.tanh)

        l_pool2 = lasagne.layers.MaxPool2DLayer(l_conv2, pool_size=(2, 2))

        l_dense1 = lasagne.layers.DenseLayer(
                            l_pool2, num_units=n_hid1,
                            b=prior_b, W=prior_W,
                            nonlinearity=lasagne.nonlinearities.tanh)

#         l_dense2 = lasagne.layers.DenseLayer(
#                             l_dense1, num_units=n_hid2,
#                             b=prior_b, W=prior_W,
#                             nonlinearity=lasagne.nonlinearities.tanh)

        l_out = lasagne.layers.DenseLayer(
                            l_dense1, num_units=n_classes,
                            b=prior_b, W=prior_W,
                            nonlinearity=lasagne.nonlinearities.softmax,
        )

        prediction = lasagne.layers.get_output(l_out)
        
        out = pm.Categorical('out', prediction,
                           observed=target_var,
                           total_size=y_train.shape[0])
    
    return neural_network

In [31]:
def create_minibatch(data):
    rng = np.random.RandomState(0)
    
    while True:
        ixs = rng.randint(len(data), size=100)
        yield data[ixs]

In [42]:
def get_prediction(samples_proba):
    return mode(np.argmax(sample_proba(X_test, 500), 
                          axis=-1), axis=0).mode[0]

In [33]:
minibatch_X = pm.generator(create_minibatch(X_train))
minibatch_y = pm.generator(create_minibatch(y_train))
neural_network_minibatch = build_ann(PriorWeights(mode='b', prior='gauss', std=0.1), 
                                     PriorWeights(mode='W', prior='gauss', std=0.1),
                                     minibatch_X, minibatch_y, 
                                     input_shape, params=[64, 64, 10])

In [34]:
ADVI_ITERS = 25000
N_SAMPLES = 500

In [35]:
with neural_network_minibatch:
    inference = pm.ADVI()
    approx = pm.fit(ADVI_ITERS, method=inference)

Average Loss = 4,711.3: 100%|██████████| 25000/25000 [24:02<00:00, 17.36it/s]


In [36]:
x = T.tensor4('X')
n = T.iscalar('n')
theano.config.compute_test_value = 'off'
_sample_proba = approx.sample_node(neural_network_minibatch.out.distribution.p, 
                                   size=n,
                                   more_replacements={minibatch_X:x})
prediction = approx.apply_replacements(neural_network_minibatch.out.distribution.p, 
                                    deterministic=True, 
                                    more_replacements={minibatch_X:x})

sample_proba = theano.function([x, n], _sample_proba)
map_proba = theano.function([x], prediction)

In [37]:
y_pred_MAP = np.argmax(map_proba(X_test), axis=1)
error_under_MAP = y_pred_MAP != y_test
error_rate_under_MAP = error_under_MAP.mean()

y_preds_posterior = sample_proba(X_test, 100)
y_pred_posterior = mode(np.argmax(y_preds_posterior, axis=-1), axis=0).mode[0]
error_under_posterior = y_pred_posterior != y_test
error_rate_under_posterior = error_under_posterior.mean()

In [38]:
print('MAP                      : %f' % error_rate_under_MAP)
print('predictive posterior mode: %f' % error_rate_under_posterior)

MAP                      : 0.500000
predictive posterior mode: 0.478667


In [41]:
print np.sum(y_pred_MAP==1)
print np.sum(y_pred_MAP==0)
print y_pred_MAP.shape[0]

18
1482
1500
